# NHiTS Trade Forecasting - Colab Setup

이 노트북은 Google Colab에서 NHiTS 예측 파이프라인을 실행하기 위한 설정입니다.


## 1. 저장소 클론 및 디렉토리 구조 생성


In [ ]:
# 저장소 클론 (GitHub에 업로드한 경우)
# !git clone <your-repo-url>
# %cd <repo-name>

# 또는 직접 파일 업로드 후 디렉토리 구조 생성
import os
from pathlib import Path

# 작업 디렉토리 설정
WORK_DIR = Path("/content/NHiTS")
WORK_DIR.mkdir(exist_ok=True)
%cd {WORK_DIR}

# 필요한 디렉토리 생성
os.makedirs("data", exist_ok=True)
os.makedirs("models", exist_ok=True)
os.makedirs("results", exist_ok=True)

print(f"작업 디렉토리: {WORK_DIR}")


## 2. 데이터 파일 업로드

아래 셀을 실행하여 데이터 파일을 업로드하세요.

필요한 파일:
- 8개 타겟 CSV: China_Export.csv, China_Import.csv, Korea_Export.csv, Korea_Import.csv, Taiwan_Export.csv, Taiwan_Import.csv, World_Export.csv, World_Import.csv
- 외생변수 CSV: tariff.csv, USD_KRW.csv, USD_CNY.csv, USD_TWD.csv, WTI.csv, copper.csv


In [ ]:
from google.colab import files
import shutil

# 파일 업로드 (여러 파일 선택 가능)
uploaded = files.upload()

# 업로드된 파일을 data/ 폴더로 이동
for filename in uploaded.keys():
    if filename.endswith('.csv'):
        shutil.move(filename, f"data/{filename}")
        print(f"Moved {filename} to data/")
    else:
        print(f"Skipped {filename} (not a CSV file)")


## 3. 패키지 설치


In [ ]:
# requirements.txt가 있다면
!pip install -r requirements.txt

# 또는 직접 설치
# !pip install neuralforecast torch pandas numpy scikit-learn matplotlib pydantic tqdm


## 4. GPU 확인 (선택사항)


In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")


## 5. 파이프라인 실행


In [ ]:
# 전체 파이프라인 실행 (학습 + 평가 + 예측)
!python -m src.pipeline.run_all


또는 개별 실행:


In [ ]:
# 학습만
# !python -m src.pipeline.train

# 평가만
# !python -m src.pipeline.evaluate

# 예측만
# !python -m src.pipeline.forecast


## 6. 결과 다운로드


In [ ]:
from google.colab import files
import zipfile

# results 폴더를 zip으로 압축
with zipfile.ZipFile('results.zip', 'w') as zipf:
    for root, dirs, files_list in os.walk('results'):
        for file in files_list:
            zipf.write(os.path.join(root, file))

# 다운로드
files.download('results.zip')

print("results.zip 다운로드 완료!")
